In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imsave
from skimage.io import imread_collection, imread
from skimage.color import rgb2gray

# Features

In [ ]:
def gray_255_scale(image):  
    arr = np.zeros(image.shape)
    for i in range(len(arr)):
        for j in range(len(arr[i])):
            arr[i][j] = int(round(image[i,j] * 255))
    return arr

In [ ]:
def homogeneity(image):
    soma = 0
    for i in range(len(image)):
        for j in range(len(image[i])):
            soma+=(image[i][j]/(1+((i-j)**2)))
    
    return soma

def variance(image):
    i_sum = 0
    j_sum = 0

    mean = np.mean(image)

    for i in range(len(image)):
        j_sum = 0
        for j in range(len(image[i])):
            j_sum += ((1-mean)**2)*image[i,j]
        i_sum+=j_sum

    return i_sum

def uniformity(image):
    sum_index_values = 0
    for i in image:
        for j in i:
            sum_index_values += j**2
    return sum_index_values

def entropy(image):
    sum_index_values = 0
    for i in image:
        for j in i:
            if j > 0:
                sum_index_values += j*np.log10(j)
    return -(sum_index_values)

def correlation(image):
    sum_index_values = 0

    for i in range(len(image)):
        for j in range(len(image[i])):
            if image[i,j] != 0:
                sum_index_values += (i - np.mean(image[i])) * (j - np.mean(image[:,j])) * image[i,j]/(np.std(image[i])*np.std(image[:,j]))
    return sum_index_values

def contrast(image):
    sum_index_values = 0
    
    for i in range(len(image)):
        for j in range(len(image[i])):
            sum_index_values += ((i - j) ** 2) * image[i,j]
    return sum_index_values

# Matriz de Co-ocorrência GLCM

In [ ]:
def glcm(image, angle, distance, normalize=False):

    lines, columns = image.shape

    glcm_matrix = []
    glcm_lines=0
    glcm_columns=0
    
    if(angle==0):
        for i in range(lines):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i][j+distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour         
        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(lines):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i][j+distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

    if(angle == 45):
        for i in range(distance, lines, 1):
            for j in range(distance, columns, 1):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j-distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour         

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(distance, columns, 1):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j-distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

    if(angle == 90):
        for i in range(distance, lines, 1):
            for j in range(columns):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(columns):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix
    
    if(angle == 135):
        for i in range(distance, lines, 1):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j+distance])
                if(reference > glcm_lines):
                    glcm_lines = reference
                if(neighbour > glcm_columns):
                    glcm_columns = neighbour

        for i in range(glcm_lines+1):
            lista = []
            for j in range(glcm_columns+1):
                lista.append(0)
            glcm_matrix.append(lista)

        for i in range(distance, lines, 1):
            for j in range(columns-distance):
                reference = int(image[i][j])
                neighbour = int(image[i-distance][j+distance])
                glcm_matrix[reference][neighbour]+=1
        if normalize == True:
            return (glcm_matrix - np.min(glcm_matrix))/np.ptp(glcm_matrix)
        return glcm_matrix

# Função de Exportação do Resultado para Arquivo CSV
Os arquivos CSV serão salvos com o seguinte: data + _ configuração de ângulo e distância utilizados (ex.: data_a0d1.csv)

In [ ]:
def exporta_csv(dataframe, angle, distance):
    df_result = pd.DataFrame(dataframe)
    nome_csv = 'data_a' + str(angle) + '_d' + str(distance) + '.csv'
    df_result.to_csv(nome_csv)

# Descritor GLCM

In [ ]:
#Diretório raiz das imagens
DIR = '/home/anatielsantos/anaconda3/Visao_computacional/Reconhecimento_Padroes/visao_trabalho_final/GLCM e extraçao de caracteristicas/images/red'
#DIR = '/home/anatielsantos/anaconda3/Visao_computacional/Reconhecimento_Padroes/visao_trabalho_final/GLCM e extraçao de caracteristicas/Base_reduzida/Training_data'

In [ ]:
def descritor_glcm(diretorio, angle, distance):    
    glcm_images = []
    sub_diretorios = os.listdir(diretorio)
    for i in range(len(sub_diretorios)):
        imagens_dir = diretorio + '/' + sub_diretorios[i] + '/*.png'
        images = imread_collection(imagens_dir)
        
        #definição da classe da imagem
        classe = i
        
        for i in range(len(images)):
            print("Imagem {}...".format(i+1))
            gray_im = rgb2gray(images[i])
            matriz_glcm = glcm(gray_255_scale(gray_im), angle, distance, normalize=True)
            glcm_images.append([uniformity(np.array(matriz_glcm)), entropy(np.array(matriz_glcm)), contrast(np.array(matriz_glcm)), correlation(np.array(matriz_glcm)), variance(np.array(matriz_glcm)), homogeneity(np.array(matriz_glcm)), classe])    
    return glcm_images

# Setup das Configurações
O array de ângulos pode ser alterado apenas removendo itens, não é possível adicionar ângulos além dos permitiros (0, 45, 90 e 135). O array de distâncias pode ser alterado para incluir ou remover valores, o número de distâncias possíveis vai depender da base de imagens utilizada. A quantidade de arquivos CSV gerados será tamanho do array de angulos x tamanho do array de distâncias.

In [ ]:
arr_angles = [0, 45, 90, 135]
arr_distances = [1, 2, 3]

In [ ]:
def aplica_descritor_glcm(DIR):
    columns = ['uniformity', 'entropy', 'contrast', 'correlation', 'variance', 'homogeneity', 'Classe']
    for a in range(len(arr_angles)):
        for d in range(len(arr_distances)):
            data = pd.DataFrame(descritor_glcm(DIR, arr_angles[a], arr_distances[d]),columns=columns)
            exporta_csv(data, arr_angles[a], arr_distances[d])

# Extração das Características
A função aplica_descritor_glcm deve receber o diretório raiz. Neste diretório as imagens devem estar separadas em subdiretórios de acordo com a sua classificação

In [ ]:
aplica_descritor_glcm(DIR)